#### Load data

In [21]:
with open('log/IIIHTTP.log','r') as InputData:
    APPdata = InputData.readlines()
    APPdata = [i.strip().split('\t') for i in APPdata]
    #For BackdroundAPP.txt format
    #APPdata = [i.strip().split(',') for i in APPdata]
    APPdata = list(filter(lambda x:'#' not in x[0],APPdata))

In [22]:
with open('log/IIIHTTP3.log','r') as InputData2:
    Testdata = InputData2.readlines()
    Testdata = [i.strip().split('\t') for i in Testdata]
    Testdata = list(filter(lambda x:'#' not in x[0],Testdata))

In [23]:
def similarity(test_data,whiteList_data):
    score = 0
    for key in fingerprint_keys_index:
        for key_index in fingerprint_keys_index[key]:
            if test_data[key_index] in whiteList_data[key]:
                score += 1
    #if score > 0:
    #    WhiteListUpdate(test_data,whiteList_data)
    return score

In [24]:
print(APPdata[2])
def WhiteListUpdate(data):
    pass

['1521442785.193737', 'ChoauHUyYj4gYcPFf', '140.92.16.71', '51635', '140.92.66.104', '80', '2', 'POST', 'iiiwsus2.iii.org.tw', '/ClientWebService/client.asmx', '-', '1.1', 'Windows-Update-Agent/10.0.10011.16384 Client-Protocol/1.70', '107957', '713', '200', 'OK', '-', '-', '(empty)', '-', '-', '-', 'FlK9jZW2DkxulMo4b', '-', 'text/plain', 'F7EQnn1YUpcvMO9tB2', '-', '-']


In [25]:
def outgoingInfo(data):
    pass

#### Training the white list fingerprint

In [26]:
fingerprint_keys_index={'ip':[4],'host':[8],'userAgent':[12]}
trainedList = {}
for index in range(len(APPdata)):
    if APPdata[index][12].strip() != "'-'":
        label = APPdata[index][12].replace('\'','').strip()
        if label not in trainedList:
            tmp_dict = {'ip':[APPdata[index][4].replace('\'','').strip()],
                        'host':[APPdata[index][8].replace('\'','').strip()],
                        'userAgent':[label]}
            trainedList.setdefault(label)
            trainedList[label] = tmp_dict
        else:
            for key in fingerprint_keys_index:
                for key_index in fingerprint_keys_index[key]:
                    tmp_APPdata = APPdata[index][key_index].replace('\'','').strip()
                    if  tmp_APPdata not in trainedList[label][key]:
                        trainedList[label].setdefault(key,[]).append(tmp_APPdata)

In [30]:
toBeDetermined = []
NoHost =[]
for test_index in range(len(Testdata)):
    similarityScore = 0
    if Testdata[test_index][8] == '-':
        NoHost.append(Testdata[test_index])
    else:
        if Testdata[test_index][12] in trainedList:
            whiteList = trainedList[Testdata[test_index][12]]
            similarityScore = similarity(Testdata[test_index],whiteList)
        else:
            max_score = 0
            likelihoodAPP = []
            for label in trainedList:
                similarityScore = similarity(Testdata[test_index],trainedList[label])
                if similarityScore == 0:
                    toBeDetermined.append(Testdata[test_index])
                    #outgoingInfo(Testdata[test_index])
                    continue
                elif similarityScore > max_score:
                    likelihoodAPP = Testdata[test_index]
                    max_score = similarityScore
                else:
                    likelihoodAPP.append(Testdata[test_index])
            #print(similarityScore,Testdata[test_index][12],likelihoodAPP)